In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.ensemble import RandomForestClassifier 

In [2]:
df = pd.read_csv("data/train.csv") 
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


/Users/hqiu/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [4]:
median_ages = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & (df['Pclass'] == j+1)]['Age'].dropna().median()

median_ages

array([[ 35. ,  28. ,  21.5],
       [ 40. ,  30. ,  25. ]])

In [5]:
df['AgeFill'] = df['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1), 'AgeFill'] = median_ages[i,j]

In [6]:
df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(5)

,Gender,Pclass,Age,AgeFill
5,1,3,NaN,25.0
17,1,2,NaN,30.0
19,0,3,NaN,21.5
26,1,3,NaN,25.0
28,0,3,NaN,21.5


In [7]:
dummies_Pclass = pd.get_dummies(df['Pclass'], prefix= 'Pclass')
dummies_Pclass.head(5)

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [8]:
df = pd.concat([df, dummies_Pclass], axis=1)
df.drop(['Pclass'], axis=1, inplace=True)

In [9]:
df.head(3)

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender,AgeFill,Pclass_1,Pclass_2,Pclass_3
0,1,0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,22.0,0.0,0.0,1.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,38.0,1.0,0.0,0.0
2,3,1,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,26.0,0.0,0.0,1.0


In [10]:
df.describe()

,PassengerId,Survived,Age,SibSp,Parch,Fare,Gender,AgeFill,Pclass_1,Pclass_2,Pclass_3
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,29.699118,0.523008,0.381594,32.204208,0.647587,29.112424,0.242424,0.206510,0.551066
std,257.353842,0.486592,14.526497,1.102743,0.806057,49.693429,0.477990,13.304424,0.428790,0.405028,0.497665
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,NaN,0.000000,0.000000,7.910400,0.000000,21.500000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,NaN,0.000000,0.000000,14.454200,1.000000,26.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,NaN,1.000000,0.000000,31.000000,1.000000,36.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,80.000000,1.000000,1.000000,1.000000


In [11]:
df = df.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
df = df.drop(['Age'], axis=1)
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Gender         891 non-null int64
AgeFill        891 non-null float64
Pclass_1       891 non-null float64
Pclass_2       891 non-null float64
Pclass_3       891 non-null float64
dtypes: float64(5), int64(5)
memory usage: 69.7 KB


,PassengerId,Survived,SibSp,Parch,Fare,Gender,AgeFill,Pclass_1,Pclass_2,Pclass_3
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,0.523008,0.381594,32.204208,0.647587,29.112424,0.242424,0.206510,0.551066
std,257.353842,0.486592,1.102743,0.806057,49.693429,0.477990,13.304424,0.428790,0.405028,0.497665
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,0.000000,0.000000,7.910400,0.000000,21.500000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,0.000000,0.000000,14.454200,1.000000,26.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,1.000000,0.000000,31.000000,1.000000,36.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,8.000000,6.000000,512.329200,1.000000,80.000000,1.000000,1.000000,1.000000


In [12]:
import sklearn.preprocessing as preprocessing

scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(df['AgeFill'])
df['Age_scaled'] = scaler.fit_transform(df['AgeFill'], age_scale_param)
fare_scale_param = scaler.fit(df['Fare'])
df['Fare_scaled'] = scaler.fit_transform(df['Fare'], fare_scale_param)

df = df.drop(['Fare', 'AgeFill'], axis=1) 

/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

In [13]:
df.head(5)

,PassengerId,Survived,SibSp,Parch,Gender,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,1,0,1,0,1,0.0,0.0,1.0,-0.534891,-0.502445
1,2,1,1,0,0,1.0,0.0,0.0,0.668392,0.786845
2,3,1,0,0,0,0.0,0.0,1.0,-0.234070,-0.488854
3,4,1,1,0,0,1.0,0.0,0.0,0.442776,0.420730
4,5,0,0,0,1,0.0,0.0,1.0,0.442776,-0.486337


In [14]:
train_data = df.values
print train_data

[[  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
   -5.34891163e-01  -5.02445171e-01]
 [  2.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    6.68391756e-01   7.86845294e-01]
 [  3.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
   -2.34070433e-01  -4.88854258e-01]
 ..., 
 [  8.89000000e+02   0.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
   -5.72493754e-01  -1.76263239e-01]
 [  8.90000000e+02   1.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
   -2.34070433e-01  -4.43810379e-02]
 [  8.91000000e+02   0.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    2.17160662e-01  -4.92377828e-01]]


In [15]:
df_predict = pd.read_csv("data/test.csv")
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [16]:
df_predict['Gender'] = df_predict['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [17]:
df_predict['AgeFill'] = df_predict['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df_predict.loc[ (df_predict.Age.isnull()) & (df_predict.Gender == i) & (df_predict.Pclass == j+1), \
                       'AgeFill'] = median_ages[i,j]

In [18]:
df_predict.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender,AgeFill
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1,34.5
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0,47.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1,62.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0,22.0


In [19]:
print df_predict['Fare'].dropna().median()

14.4542


In [20]:
print df_predict[df_predict.Fare.isnull()]

     PassengerId  Pclass                Name   Sex   Age  SibSp  Parch Ticket  \
152         1044       3  Storey, Mr. Thomas  male  60.5      0      0   3701   

     Fare Cabin Embarked  Gender  AgeFill  
152   NaN   NaN        S       1     60.5  


In [21]:
df_predict.loc[ df_predict['Fare'].isnull(), 'Fare' ] = df_predict['Fare'].dropna().median()

In [22]:
scaler = preprocessing.StandardScaler()
age_scale_param = scaler.fit(df_predict['AgeFill'])
df_predict['Age_scaled'] = scaler.fit_transform(df_predict['AgeFill'], age_scale_param)
fare_scale_param = scaler.fit(df_predict['Fare'])
df_predict['Fare_scaled'] = scaler.fit_transform(df_predict['Fare'], fare_scale_param)

df_predict = df_predict.drop(['Fare', 'AgeFill'], axis=1) 

/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/hqiu/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

In [23]:
dummies_Pclass = pd.get_dummies(df_predict['Pclass'], prefix= 'Pclass')
df_predict = pd.concat([df_predict, dummies_Pclass], axis=1)
df_predict.drop(['Pclass'], axis=1, inplace=True)

In [24]:
df_predict.describe()
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 15 columns):
PassengerId    418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Cabin          91 non-null object
Embarked       418 non-null object
Gender         418 non-null int64
Age_scaled     418 non-null float64
Fare_scaled    418 non-null float64
Pclass_1       418 non-null float64
Pclass_2       418 non-null float64
Pclass_3       418 non-null float64
dtypes: float64(6), int64(4), object(5)
memory usage: 49.1+ KB


In [25]:
df_predict = df_predict.drop(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
df_predict = df_predict.drop(['Age'], axis=1)
df_predict.describe()
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
PassengerId    418 non-null int64
SibSp          418 non-null int64
Parch          418 non-null int64
Gender         418 non-null int64
Age_scaled     418 non-null float64
Fare_scaled    418 non-null float64
Pclass_1       418 non-null float64
Pclass_2       418 non-null float64
Pclass_3       418 non-null float64
dtypes: float64(5), int64(4)
memory usage: 29.5 KB


In [26]:
test_data = df_predict.values
print test_data

[[  8.92000000e+02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  8.93000000e+02   1.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  8.94000000e+02   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    1.00000000e+00   0.00000000e+00]
 ..., 
 [  1.30700000e+03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  1.30800000e+03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  1.30900000e+03   1.00000000e+00   1.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]]


In [36]:
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 200)

# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train_data[0::,2::],train_data[0::,1])

In [28]:
forest.score(train_data[0::,2::],train_data[0::,1])

0.97979797979797978

In [29]:
# Take the same decision trees and run it on the test data
y_predict = forest.predict(test_data[0::,1::]).astype(int)

In [30]:
print y_predict

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [33]:
from pandas import DataFrame

output_df = DataFrame({'PassengerId': test_data[0::,0].astype(int), 'Survived': y_predict})

In [34]:
output_df.to_csv("data/output/randomforest_example.csv", index=False) 